In [14]:
from IPython import display
from ipywidgets import widgets
from main import*
from time import sleep
from pynq.iop import *
from bottle import *
from drinks import * 
from enum import Enum

def stage_1 (change):
    if (change['new']):
        print("ON")
        html.value="<b>STAGE 1 </b>"
        drop.options = name
        drop.observe(stage_2,'value')
            
    else:
        print("RESET")
        
        
    
    return
#### -----------------------------------------------------------

def stage_2(change):
    global typeOfDrink
    if change['type'] == 'change' and change['name'] == 'value':
        html.value="<b>STAGE 2</b>"
        i = 0
        for d in drinks:
            if (change['new'] == d.getDrinkName() ):
                typeOfDrink = d
                print (typeOfDrink.drinkName)
                print (typeOfDrink.getIngredients())
                startButton.disabled=False
                startButton.description='START DRINK'
                startButton.button_style='danger'
                startButton.on_click(stage_3)
                return
            else:
                i+=1
                
    return

#### -----------------------------------------------------------
def stage_3(change):
    global typeOfDrink
    startButton.disabled=True
    html.value="<b>STAGE 3</b>"
    text.value="Pouring drink now..."
    makeDrink()
    print(typeOfDrink.drinkName + " is ready for pickup!")
    typeOfDrink = None
    return
#### -----------------------------------------------------------
def checkLiquids(change):
    startButton.disabled=True
#     text.value="later"
    for b in bottleDictionary:
         print(bottleDictionary[b].getInfo())    

    return
#### -----------------------------------------------------------
typeOfDrink = None
name = []
toggle = widgets.ToggleButton()
valid = widgets.Valid()
drop = widgets.Dropdown()
startButton = widgets.Button()
bottleLevel = widgets.Button()
html = widgets.HTML()
text = widgets.Label()
finish = widgets.Label()

#-------Hardware-Globals------#
# Name of Drink file to load from
#drink_file = sys.argv[1]
drinkFile = "drinkList.txt"

# Define Drink States
class State(Enum):
    ORDER = 1
    PREPARE = 2
    DISPENSE = 3
    PICKUP = 4

# Load drinks
ingredients = [] 
drinks = []
#Average flow-rate of each bottle Oz/s
drinkConstant = [0.34, 0.343, 0.348, 0.365]
bottleSize = [16.9 for i in range(0,4)]
#maps ingredient name to bottle
bottleDictionary = dict()
# Declare Drink State
machineState = State(1)
basePin = 0
valve = [Arduino_IO(3, (basePin+i), 'out') for i in range (4)]

#------Hardware Methods--------#
# Load text file into different objects
def loadFile(fname):
    global ingredients, drinks
    count = 0
    with open(fname, 'r') as drinkFile:
        for line in drinkFile.readlines():
            lines = line.rstrip()
            #change to pointer for some reason
            lines.rstrip()
            if count == 0:
                #all ingredients on the machine
                ingredients = lines.split('\t')
                declareBottles(ingredients)
            else:
                #get drink names
                args = lines.split('\t')
                #add ingredients to the drink
                if line[0] != '\t':
                    drinks.append(Drink(args[0]))
                else: 
                    del args[0]
                    drinks[len(drinks)-1].addIngredient(args[0], args[1])
                    #print(args)
            count += 1
            
#load bottles from text file
# bottle name -> bottle
def declareBottles(ingredients):
    global drinkConstant, bottleSize
    index = 0
    for i in ingredients:
        bottleDictionary[i] = Bottle(index, i, bottleSize[index], drinkConstant[index])
        index += 1
        
def makeDrink():
    # iterate through the ingredients
    # go one ingredient at a time and also the amount
    # locate the bottle index (which vale to turn on)
    global typeOfDrink, machineState, bottleDictionary
    for bottleName in typeOfDrink.drinkList:
        # get the bottle object
        bottle = bottleDictionary[bottleName]
        # get the amount to dispense
        amount = float(typeOfDrink.drinkList[bottleName])
        k = float(bottle.sleepConstant)
        sleepTime = amount/k
        print(str(sleepTime)+" s")
        bottleIndex = bottle.bottleNum
        bottle.dispense(amount)
        #turn on the valve
        valve[bottleIndex].write(1)
        #sleep certian num of seconds
        sleep(sleepTime)
        #turn off the valve
        valve[bottleIndex].write(0)
        #delay between valves
        sleep(0.1)
#------------------#


#### INITIAL ----------------------------------------------------
toggle.description = "TAP TO GET LIT"
#---------------------------
bottleLevel.disabled=False
bottleLevel.description='BOTTLE LEVEL'
bottleLevel.button_style='warning'
#---------------------------
loadFile(drinkFile)
name = []
name.append("CHOOSE YOUR DRINK")
for d in drinks:
    name.append(d.getDrinkName())

container2 = widgets.HBox([startButton,bottleLevel])
container_button = widgets.VBox([html,toggle,drop,container2,text,finish])
toggle.observe(stage_1,'value')
bottleLevel.on_click(checkLiquids)
display.display(container_button)